In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00
  

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

import os
import json
import math
import random
import pandas as pd
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, fbeta_score, precision_recall_fscore_support

In [ ]:
main_path = "/content/drive/MyDrive/VU Thesis/Code/fine_tune"
data_path = "/content/drive/MyDrive/VU Thesis/Code/fine_tune/data"

In [ ]:
# Custom classification report function to use F0.5 score

from sklearn.metrics import precision_score, recall_score, fbeta_score, precision_recall_fscore_support
from sklearn.utils import _unique
from sklearn.metrics import _classification
from sklearn.utils import multiclass
import warnings
import numpy as np
from numbers import Integral, Real
from sklearn.utils._param_validation import (
    Hidden,
    Interval,
    Options,
    StrOptions,
    validate_params,
)

@validate_params(
    {
        "y_true": ["array-like", "sparse matrix"],
        "y_pred": ["array-like", "sparse matrix"],
        "labels": ["array-like", None],
        "target_names": ["array-like", None],
        "sample_weight": ["array-like", None],
        "digits": [Interval(Integral, 0, None, closed="left")],
        "output_dict": ["boolean"],
        "zero_division": [
            Options(Real, {0.0, 1.0}),
            "nan",
            StrOptions({"warn"}),
        ],
    },
    prefer_skip_nested_validation=True,
)
def classification_report_f0_5(
    y_true,
    y_pred,
    *,
    labels=None,
    target_names=None,
    sample_weight=None,
    digits=2,
    output_dict=False,
    zero_division="warn",
):
    """Build a text report showing the main classification metrics.

    Read more in the :ref:`User Guide <classification_report>`.

    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) target values.

    y_pred : 1d array-like, or label indicator array / sparse matrix
        Estimated targets as returned by a classifier.

    labels : array-like of shape (n_labels,), default=None
        Optional list of label indices to include in the report.

    target_names : array-like of shape (n_labels,), default=None
        Optional display names matching the labels (same order).

    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.

    digits : int, default=2
        Number of digits for formatting output floating point values.
        When ``output_dict`` is ``True``, this will be ignored and the
        returned values will not be rounded.

    output_dict : bool, default=False
        If True, return output as dict.

        .. versionadded:: 0.20

    zero_division : {"warn", 0.0, 1.0, np.nan}, default="warn"
        Sets the value to return when there is a zero division. If set to
        "warn", this acts as 0, but warnings are also raised.

        .. versionadded:: 1.3
           `np.nan` option was added.

    Returns
    -------
    report : str or dict
        Text summary of the precision, recall, F1 score for each class.
        Dictionary returned if output_dict is True. Dictionary has the
        following structure::

            {'label 1': {'precision':0.5,
                         'recall':1.0,
                         'f1-score':0.67,
                         'support':1},
             'label 2': { ... },
              ...
            }

        The reported averages include macro average (averaging the unweighted
        mean per label), weighted average (averaging the support-weighted mean
        per label), and sample average (only for multilabel classification).
        Micro average (averaging the total true positives, false negatives and
        false positives) is only shown for multi-label or multi-class
        with a subset of classes, because it corresponds to accuracy
        otherwise and would be the same for all metrics.
        See also :func:`precision_recall_fscore_support` for more details
        on averages.

        Note that in binary classification, recall of the positive class
        is also known as "sensitivity"; recall of the negative class is
        "specificity".

    See Also
    --------
    precision_recall_fscore_support: Compute precision, recall, F-measure and
        support for each class.
    confusion_matrix: Compute confusion matrix to evaluate the accuracy of a
        classification.
    multilabel_confusion_matrix: Compute a confusion matrix for each class or sample.

    Examples
    --------
    >>> from sklearn.metrics import classification_report
    >>> y_true = [0, 1, 2, 2, 2]
    >>> y_pred = [0, 0, 2, 2, 1]
    >>> target_names = ['class 0', 'class 1', 'class 2']
    >>> print(classification_report(y_true, y_pred, target_names=target_names))
                  precision    recall  f1-score   support
    <BLANKLINE>
         class 0       0.50      1.00      0.67         1
         class 1       0.00      0.00      0.00         1
         class 2       1.00      0.67      0.80         3
    <BLANKLINE>
        accuracy                           0.60         5
       macro avg       0.50      0.56      0.49         5
    weighted avg       0.70      0.60      0.61         5
    <BLANKLINE>
    >>> y_pred = [1, 1, 0]
    >>> y_true = [1, 1, 1]
    >>> print(classification_report(y_true, y_pred, labels=[1, 2, 3]))
                  precision    recall  f1-score   support
    <BLANKLINE>
               1       1.00      0.67      0.80         3
               2       0.00      0.00      0.00         0
               3       0.00      0.00      0.00         0
    <BLANKLINE>
       micro avg       1.00      0.67      0.80         3
       macro avg       0.33      0.22      0.27         3
    weighted avg       1.00      0.67      0.80         3
    <BLANKLINE>
    """

    y_true, y_pred = _unique.attach_unique(y_true, y_pred)
    y_type, y_true, y_pred = _classification._check_targets(y_true, y_pred)

    if labels is None:
        labels = multiclass.unique_labels(y_true, y_pred)
        labels_given = False
    else:
        labels = np.asarray(labels)
        labels_given = True

    # labelled micro average
    micro_is_accuracy = (y_type == "multiclass" or y_type == "binary") and (
        not labels_given or (set(labels) >= set(multiclass.unique_labels(y_true, y_pred)))
    )

    if target_names is not None and len(labels) != len(target_names):
        if labels_given:
            warnings.warn(
                "labels size, {0}, does not match size of target_names, {1}".format(
                    len(labels), len(target_names)
                )
            )
        else:
            raise ValueError(
                "Number of classes, {0}, does not match size of "
                "target_names, {1}. Try specifying the labels "
                "parameter".format(len(labels), len(target_names))
            )
    if target_names is None:
        target_names = ["%s" % l for l in labels]

    headers = ["precision", "recall", "f0.5-score", "support"]
    # compute per-class results without averaging
    p, r, f1, s = precision_recall_fscore_support(
        y_true,
        y_pred,
        labels=labels,
        beta=0.5,
        average=None,
        sample_weight=sample_weight,
        zero_division=zero_division,
    )
    rows = zip(target_names, p, r, f1, s)

    if y_type.startswith("multilabel"):
        average_options = ("micro", "macro", "weighted", "samples")
    else:
        average_options = ("micro", "macro", "weighted")

    if output_dict:
        report_dict = {label[0]: label[1:] for label in rows}
        for label, scores in report_dict.items():
            report_dict[label] = dict(zip(headers, [float(i) for i in scores]))
    else:
        longest_last_line_heading = "weighted avg"
        name_width = max(len(cn) for cn in target_names)
        width = max(name_width, len(longest_last_line_heading), digits)
        head_fmt = "{:>{width}s} " + " {:>9}" * len(headers)
        report = head_fmt.format("", *headers, width=width)
        report += "\n\n"
        row_fmt = "{:>{width}s} " + " {:>9.{digits}f}" * 3 + " {:>9}\n"
        for row in rows:
            report += row_fmt.format(*row, width=width, digits=digits)
        report += "\n"

    # compute all applicable averages
    for average in average_options:
        if average.startswith("micro") and micro_is_accuracy:
            line_heading = "accuracy"
        else:
            line_heading = average + " avg"

        # compute averages with specified averaging method
        avg_p, avg_r, avg_f1, _ = precision_recall_fscore_support(
            y_true,
            y_pred,
            labels=labels,
            beta=0.5,
            average=average,
            sample_weight=sample_weight,
            zero_division=zero_division,
        )
        avg = [avg_p, avg_r, avg_f1, np.sum(s)]

        if output_dict:
            report_dict[line_heading] = dict(zip(headers, [float(i) for i in avg]))
        else:
            if line_heading == "accuracy":
                row_fmt_accuracy = (
                    "{:>{width}s} "
                    + " {:>9.{digits}}" * 2
                    + " {:>9.{digits}f}"
                    + " {:>9}\n"
                )
                report += row_fmt_accuracy.format(
                    line_heading, "", "", *avg[2:], width=width, digits=digits
                )
            else:
                report += row_fmt.format(line_heading, *avg, width=width, digits=digits)

    if output_dict:
        if "accuracy" in report_dict.keys():
            report_dict["accuracy"] = report_dict["accuracy"]["precision"]
        return report_dict
    else:
        return report

# FULL DATA

In [ ]:
print("loading data...")

print("loading train_data.csv")
train_df =pd.read_csv(os.path.join(data_path, "train_data.csv"))

print("loading dev_data.csv")
dev_df =pd.read_csv(os.path.join(data_path, "dev_data.csv"))

print("loading data compelete.")

loading data...
loading train_data.csv
loading dev_data.csv
loading data compelete.


### Epoch 1

In [ ]:
# Define model arguments
model_args = ClassificationArgs(eval_batch_size=64,
                                evaluate_during_training=True,
                                save_eval_checkpoints=False,
                                save_steps=-1,
                                manual_seed=42,
                                evaluate_during_training_steps=2621,
                                overwrite_output_dir=True,
                                labels_list=['G', 'PREP', 'DET','SVA']
                                )
model_args.num_train_epochs = 10

# Initialize the model
model = ClassificationModel('roberta',
                            '/content/drive/MyDrive/VU Thesis/Code/baby_models/V20L4/model_e1_v20_l4',
                            args=model_args,
                            num_labels=4
                            )


f05_scores_e1 = []
epoch_num_count = 1
_last_metric_str = None  # Store last metric as a string for comparison

def custom_metric(preds, labels):
    global _last_metric_str
    global epoch_num_count
    metric = classification_report_f0_5(labels, preds, output_dict=True)

    # Serialize to string for reliable comparison
    metric_str = json.dumps(metric, sort_keys=True)

    if metric_str != _last_metric_str:
        f05_scores_e1.append({f"epoch{epoch_num_count}":metric})
        epoch_num_count+=1
        _last_metric_str = metric_str

    return metric['macro avg']['f0.5-score']

# Train the model with the callback
model.train_model(train_df, eval_df=dev_df, compute_metrics=custom_metric)

# f05_scores_e1_df = pd.DataFrame(f05_scores_e1)
# print(f05_scores_e1_df)
for item in f05_scores_e1:
    print(item)

with open(os.path.join(main_path, 'epoch_1_evals.json'), 'w') as out_file:
    json.dump(f05_scores_e1, out_file)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/VU Thesis/Code/baby_models/V20L4/model_e1_v20_l4 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/41 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 10:   0%|          | 0/2621 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'epoch1': {'0': {'precision': 0.5695477962220951, 'recall': 0.6017538554581191, 'f0.5-score': 0.5757102354915234, 'support': 3307.0}, '1': {'precision': 0.9020501138952164, 'recall': 0.591044776119403, 'f0.5-score': 0.8161582852431987, 'support': 1340.0}, '2': {'precision': 0.7567925778661365, 'recall': 0.5947916666666667, 'f0.5-score': 0.7176973353443942, 'support': 1920.0}, '3': {'precision': 0.2086720867208672, 'recall': 0.5486935866983373, 'f0.5-score': 0.23819344194679315, 'support': 421.0}, 'accuracy': 0.5945907269605037, 'macro avg': {'precision': 0.6092656436760788, 'recall': 0.5840709712356316, 'f0.5-score': 0.5869398245064774, 'support': 6988.0}, 'weighted avg': {'precision': 0.6630129382854216, 'recall': 0.5945907269605037, 'f0.5-score': 0.6404957318141372, 'support': 6988.0}}}
{'epoch2': {'0': {'precision': 0.754722381224957, 'recall': 0.8408801020408163, 'f0.5-score': 0.7705119214586256, 'support': 3136.0}, '1': {'precision': 0.8929384965831435, 'recall': 0.58903080390683

In [ ]:
for item in f05_scores_e1:
    print(item)

{'epoch1': {'0': {'precision': 0.5695477962220951, 'recall': 0.6017538554581191, 'f0.5-score': 0.5757102354915234, 'support': 3307.0}, '1': {'precision': 0.9020501138952164, 'recall': 0.591044776119403, 'f0.5-score': 0.8161582852431987, 'support': 1340.0}, '2': {'precision': 0.7567925778661365, 'recall': 0.5947916666666667, 'f0.5-score': 0.7176973353443942, 'support': 1920.0}, '3': {'precision': 0.2086720867208672, 'recall': 0.5486935866983373, 'f0.5-score': 0.23819344194679315, 'support': 421.0}, 'accuracy': 0.5945907269605037, 'macro avg': {'precision': 0.6092656436760788, 'recall': 0.5840709712356316, 'f0.5-score': 0.5869398245064774, 'support': 6988.0}, 'weighted avg': {'precision': 0.6630129382854216, 'recall': 0.5945907269605037, 'f0.5-score': 0.6404957318141372, 'support': 6988.0}}}
{'epoch2': {'0': {'precision': 0.754722381224957, 'recall': 0.8408801020408163, 'f0.5-score': 0.7705119214586256, 'support': 3136.0}, '1': {'precision': 0.8929384965831435, 'recall': 0.58903080390683

In [ ]:
for item in f05_scores_e1:
  for k , v in item.items():
    print(k,v['macro avg']['f0.5-score'])

epoch1 0.5869398245064774
epoch2 0.8035347774653994
epoch3 0.8690032260433115
epoch4 0.8792684755627849
epoch5 0.8645069805680621
epoch6 0.8821109348523397
epoch7 0.8689473962820986
epoch8 0.8309469950106507
epoch9 0.8309543708959224
epoch10 0.841388851916296


### Epoch 10

In [ ]:
# Define model arguments
model_args = ClassificationArgs(eval_batch_size=64,
                                # train_batch_size=64,
                                evaluate_during_training=True,
                                save_eval_checkpoints=False,
                                save_steps=-1,
                                manual_seed=42,
                                evaluate_during_training_steps=2621,
                                overwrite_output_dir=True,
                                labels_list=['G', 'PREP', 'DET','SVA']
                                )
model_args.num_train_epochs = 10

# Initialize the model
model = ClassificationModel('roberta',
                            '/content/drive/MyDrive/VU Thesis/Code/baby_models/V20L4/model_e10_v20_l4',
                            args=model_args,
                            num_labels=4
                            )


f05_scores_e10 = []
epoch_num_count = 1
_last_metric_str = None  # Store last metric as a string for comparison

def custom_metric(preds, labels):
    global _last_metric_str
    global epoch_num_count
    metric = classification_report_f0_5(labels, preds, output_dict=True)

    # Serialize to string for reliable comparison
    metric_str = json.dumps(metric, sort_keys=True)

    if metric_str != _last_metric_str:
        f05_scores_e10.append({f"epoch{epoch_num_count}":metric})
        epoch_num_count+=1
        _last_metric_str = metric_str

    return metric['macro avg']['f0.5-score']

# Train the model with the callback
model.train_model(train_df, eval_df=dev_df, compute_metrics=custom_metric)

# f05_scores_e10_df = pd.DataFrame(f05_scores_e10)
# print(f05_scores_e10_df)
for item in f05_scores_e10:
    print(item)

with open(os.path.join(main_path, 'epoch_10_evals.json'), 'w') as out_file:
    json.dump(f05_scores_e10, out_file)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/VU Thesis/Code/baby_models/V20L4/model_e10_v20_l4 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/41 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 10:   0%|          | 0/2621 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'epoch1': {'0': {'precision': 0.8205495134516314, 'recall': 0.7659631311781993, 'f0.5-score': 0.8090185676392573, 'support': 3743.0}, '1': {'precision': 0.9214123006833713, 'recall': 0.593108504398827, 'f0.5-score': 0.8295734208367515, 'support': 1364.0}, '2': {'precision': 1.0, 'recall': 0.9914586070959264, 'f0.5-score': 0.9982799682455676, 'support': 1522.0}, '3': {'precision': 0.26919602529358627, 'recall': 0.83008356545961, 'f0.5-score': 0.3112596615834552, 'support': 359.0}, 'accuracy': 0.7846307956496852, 'macro avg': {'precision': 0.7527894598571473, 'recall': 0.7951534520331407, 'f0.5-score': 0.7370329045762579, 'support': 6988.0}, 'weighted avg': {'precision': 0.8509966485492233, 'recall': 0.7846307956496852, 'f0.5-score': 0.8286804486080829, 'support': 6988.0}}}
{'epoch2': {'0': {'precision': 0.7936462507155123, 'recall': 0.948034188034188, 'f0.5-score': 0.8203656588367553, 'support': 2925.0}, '1': {'precision': 0.9555808656036446, 'recall': 0.5929328621908128, 'f0.5-score':

In [ ]:
for item in f05_scores_e10:
    print(item)

{'epoch1': {'0': {'precision': 0.8205495134516314, 'recall': 0.7659631311781993, 'f0.5-score': 0.8090185676392573, 'support': 3743.0}, '1': {'precision': 0.9214123006833713, 'recall': 0.593108504398827, 'f0.5-score': 0.8295734208367515, 'support': 1364.0}, '2': {'precision': 1.0, 'recall': 0.9914586070959264, 'f0.5-score': 0.9982799682455676, 'support': 1522.0}, '3': {'precision': 0.26919602529358627, 'recall': 0.83008356545961, 'f0.5-score': 0.3112596615834552, 'support': 359.0}, 'accuracy': 0.7846307956496852, 'macro avg': {'precision': 0.7527894598571473, 'recall': 0.7951534520331407, 'f0.5-score': 0.7370329045762579, 'support': 6988.0}, 'weighted avg': {'precision': 0.8509966485492233, 'recall': 0.7846307956496852, 'f0.5-score': 0.8286804486080829, 'support': 6988.0}}}
{'epoch2': {'0': {'precision': 0.7936462507155123, 'recall': 0.948034188034188, 'f0.5-score': 0.8203656588367553, 'support': 2925.0}, '1': {'precision': 0.9555808656036446, 'recall': 0.5929328621908128, 'f0.5-score':

In [ ]:
for item in f05_scores_e10:
  for k , v in item.items():
    print(k,v['macro avg']['f0.5-score'])

epoch1 0.7370329045762579
epoch2 0.8906185760119897
epoch3 0.9090168753864079
epoch4 0.8958703090052436
epoch5 0.9056979753954882
epoch6 0.8896873347700005
epoch7 0.8799197755646971
epoch8 0.8439704296950851
epoch9 0.8528419464543615
epoch10 0.845804050758941


# HALF of DATA

In [ ]:
print("loading data...")

print("loading train_data.csv")
train_df =pd.read_csv(os.path.join(data_path, "train_data.csv"))

print("loading dev_data.csv")
dev_df =pd.read_csv(os.path.join(data_path, "dev_data.csv"))

print("loading data compelete.")

loading data...
loading train_data.csv
loading dev_data.csv
loading data compelete.


In [ ]:
train_df, tmp = train_test_split(train_df, train_size=0.5, random_state=42, stratify=train_df['labels'])

### Epoch 1

In [ ]:
# Define model arguments
model_args = ClassificationArgs(eval_batch_size=64,
                                evaluate_during_training=True,
                                save_eval_checkpoints=False,
                                save_steps=-1,
                                evaluate_during_training_steps=1311,
                                overwrite_output_dir=True,
                                labels_list=['G', 'PREP', 'DET','SVA']
                                )
model_args.num_train_epochs = 10

# Initialize the model
model = ClassificationModel('roberta',
                            '/content/drive/MyDrive/VU Thesis/Code/baby_models/V20L4/model_e1_v20_l4',
                            args=model_args,
                            num_labels=4
                            )


f05_scores_e1 = []
epoch_num_count = 1
_last_metric_str = None  # Store last metric as a string for comparison

def custom_metric(preds, labels):
    global _last_metric_str
    global epoch_num_count
    metric = classification_report_f0_5(labels, preds, output_dict=True)

    # Serialize to string for reliable comparison
    metric_str = json.dumps(metric, sort_keys=True)

    if metric_str != _last_metric_str:
        f05_scores_e1.append({f"epoch{epoch_num_count}":metric})
        epoch_num_count+=1
        _last_metric_str = metric_str

    return metric['macro avg']['f0.5-score']

# Train the model with the callback
model.train_model(train_df, eval_df=dev_df, compute_metrics=custom_metric)

# f05_scores_e1_df = pd.DataFrame(f05_scores_e1)
# print(f05_scores_e1_df)
for item in f05_scores_e1:
    print(item)

with open(os.path.join(main_path, 'epoch_1_evals_half.json'), 'w') as out_file:
    json.dump(f05_scores_e1, out_file)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/VU Thesis/Code/baby_models/V20L4/model_e1_v20_l4 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 10:   0%|          | 0/1311 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'epoch1': {'0': {'precision': 0.5117344018317115, 'recall': 0.5958013995334889, 'f0.5-score': 0.5265948047358191, 'support': 3001.0}, '1': {'precision': 0.8553530751708428, 'recall': 0.5862607338017174, 'f0.5-score': 0.7834341748383059, 'support': 1281.0}, '2': {'precision': 0.7813121272365805, 'recall': 0.560361216730038, 'f0.5-score': 0.7242014742014742, 'support': 2104.0}, '3': {'precision': 0.2971996386630533, 'recall': 0.5465116279069767, 'f0.5-score': 0.32703777335984097, 'support': 602.0}, 'accuracy': 0.5791356611333714, 'macro avg': {'precision': 0.6113998107255469, 'recall': 0.5722337444930552, 'f0.5-score': 0.59031705678386, 'support': 6988.0}, 'weighted avg': {'precision': 0.637409434369167, 'recall': 0.5791356611333714, 'f0.5-score': 0.615982660026129, 'support': 6988.0}}}
{'epoch2': {'0': {'precision': 0.4358900973096737, 'recall': 0.6639058413251961, 'f0.5-score': 0.46803933620159804, 'support': 2294.0}, '1': {'precision': 0.8006833712984055, 'recall': 0.583402489626556,

In [ ]:
for item in f05_scores_e1:
    print(item)

{'epoch1': {'0': {'precision': 0.5117344018317115, 'recall': 0.5958013995334889, 'f0.5-score': 0.5265948047358191, 'support': 3001.0}, '1': {'precision': 0.8553530751708428, 'recall': 0.5862607338017174, 'f0.5-score': 0.7834341748383059, 'support': 1281.0}, '2': {'precision': 0.7813121272365805, 'recall': 0.560361216730038, 'f0.5-score': 0.7242014742014742, 'support': 2104.0}, '3': {'precision': 0.2971996386630533, 'recall': 0.5465116279069767, 'f0.5-score': 0.32703777335984097, 'support': 602.0}, 'accuracy': 0.5791356611333714, 'macro avg': {'precision': 0.6113998107255469, 'recall': 0.5722337444930552, 'f0.5-score': 0.59031705678386, 'support': 6988.0}, 'weighted avg': {'precision': 0.637409434369167, 'recall': 0.5791356611333714, 'f0.5-score': 0.615982660026129, 'support': 6988.0}}}
{'epoch2': {'0': {'precision': 0.4358900973096737, 'recall': 0.6639058413251961, 'f0.5-score': 0.46803933620159804, 'support': 2294.0}, '1': {'precision': 0.8006833712984055, 'recall': 0.583402489626556,

In [ ]:
for item in f05_scores_e1:
  for k , v in item.items():
    # print(k,v)
    print(k,v['macro avg']['f0.5-score'])

epoch1 0.59031705678386
epoch2 0.6620179109912859
epoch3 0.6903315633724443
epoch4 0.763186098734778
epoch5 0.7538632791528652
epoch6 0.791461875827684
epoch7 0.8043349759627726
epoch8 0.7604117339714573
epoch9 0.7910931308693561
epoch10 0.7836947995405191


### Epoch 10

In [ ]:
# Define model arguments
model_args = ClassificationArgs(eval_batch_size=64,
                                evaluate_during_training=True,
                                save_eval_checkpoints=False,
                                save_steps=-1,
                                manual_seed=42,
                                evaluate_during_training_steps=1311,
                                overwrite_output_dir=True,
                                labels_list=['G', 'PREP', 'DET','SVA']
                                )
model_args.num_train_epochs = 10

# Initialize the model
model = ClassificationModel('roberta',
                            '/content/drive/MyDrive/VU Thesis/Code/baby_models/V20L4/model_e10_v20_l4',
                            args=model_args,
                            num_labels=4
                            )


f05_scores_e10 = []
epoch_num_count = 1
_last_metric_str = None  # Store last metric as a string for comparison

def custom_metric(preds, labels):
    global _last_metric_str
    global epoch_num_count
    metric = classification_report_f0_5(labels, preds, output_dict=True)

    # Serialize to string for reliable comparison
    metric_str = json.dumps(metric, sort_keys=True)

    if metric_str != _last_metric_str:
        f05_scores_e10.append({f"epoch{epoch_num_count}":metric})
        epoch_num_count+=1
        _last_metric_str = metric_str

    return metric['macro avg']['f0.5-score']

# Train the model with the callback
model.train_model(train_df, eval_df=dev_df, compute_metrics=custom_metric)

# f05_scores_e10_df = pd.DataFrame(f05_scores_e10)
# print(f05_scores_e10_df)
for item in f05_scores_e10:
    print(item)

with open(os.path.join(main_path, 'epoch_10_evals_half.json'), 'w') as out_file:
    json.dump(f05_scores_e10, out_file)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/VU Thesis/Code/baby_models/V20L4/model_e10_v20_l4 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 10:   0%|          | 0/1311 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'epoch1': {'0': {'precision': 0.7650257584430452, 'recall': 0.7460228858498466, 'f0.5-score': 0.761148129164531, 'support': 3583.0}, '1': {'precision': 0.8758542141230068, 'recall': 0.5952012383900929, 'f0.5-score': 0.8003746877601998, 'support': 1292.0}, '2': {'precision': 0.9973492379058979, 'recall': 0.94180225281602, 'f0.5-score': 0.9857217710243646, 'support': 1598.0}, '3': {'precision': 0.2682926829268293, 'recall': 0.5766990291262136, 'f0.5-score': 0.3004248432126239, 'support': 515.0}, 'accuracy': 0.750429307384087, 'macro avg': {'precision': 0.7266304733496949, 'recall': 0.7149313515455432, 'f0.5-score': 0.7119173577904299, 'support': 6988.0}, 'weighted avg': {'precision': 0.8020357399870202, 'recall': 0.750429307384087, 'f0.5-score': 0.7858013777524512, 'support': 6988.0}}}
{'epoch2': {'0': {'precision': 0.7489982827704637, 'recall': 0.8682813536828136, 'f0.5-score': 0.7701589170100059, 'support': 3014.0}, '1': {'precision': 0.9009111617312073, 'recall': 0.5929535232383808, 

In [ ]:
for item in f05_scores_e10:
    print(item)

{'epoch1': {'0': {'precision': 0.7650257584430452, 'recall': 0.7460228858498466, 'f0.5-score': 0.761148129164531, 'support': 3583.0}, '1': {'precision': 0.8758542141230068, 'recall': 0.5952012383900929, 'f0.5-score': 0.8003746877601998, 'support': 1292.0}, '2': {'precision': 0.9973492379058979, 'recall': 0.94180225281602, 'f0.5-score': 0.9857217710243646, 'support': 1598.0}, '3': {'precision': 0.2682926829268293, 'recall': 0.5766990291262136, 'f0.5-score': 0.3004248432126239, 'support': 515.0}, 'accuracy': 0.750429307384087, 'macro avg': {'precision': 0.7266304733496949, 'recall': 0.7149313515455432, 'f0.5-score': 0.7119173577904299, 'support': 6988.0}, 'weighted avg': {'precision': 0.8020357399870202, 'recall': 0.750429307384087, 'f0.5-score': 0.7858013777524512, 'support': 6988.0}}}
{'epoch2': {'0': {'precision': 0.7489982827704637, 'recall': 0.8682813536828136, 'f0.5-score': 0.7701589170100059, 'support': 3014.0}, '1': {'precision': 0.9009111617312073, 'recall': 0.5929535232383808, 

In [ ]:
for item in f05_scores_e10:
  for k , v in item.items():
    print(k,v['macro avg']['f0.5-score'])

epoch1 0.7119173577904299
epoch2 0.8260227762585975
epoch3 0.8143435574424752
epoch4 0.8380354860635373
epoch5 0.8422210119242948
epoch6 0.7832795664805713
epoch7 0.8499169900421547
epoch8 0.8172135137560359
epoch9 0.8228752684361242
epoch10 0.8209056798376733
